<a href="https://www.kaggle.com/code/pietrocaforio/unimodal-ct-training-kaggle?scriptVersionId=197328030" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 208, done.
remote: Total 208 (delta 0), reused 0 (delta 0), pack-reused 208 (from 1)
Receiving objects: 100% (208/208), 3.56 MiB | 11.14 MiB/s, done.
Resolving deltas: 100% (122/122), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
!pip install wandb

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [5]:
import wandb
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

from sklearn.utils.class_weight import compute_class_weight

### Train ResNet model

In [7]:
def train(model,config, run_name=None):
    wandb.init(
        # set the wandb project where this run will be logged
        project="unimodal_ct_training",
        name = run_name,
        # track hyperparameters and run metadata
        config=config
    )
    if config["class_weights"]: 
        config["class_weights"] = torch.tensor(config["class_weights"], dtype=torch.float).to(device) 
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
    criterion = nn.CrossEntropyLoss(weight = config["class_weights"])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = config["reduce_lr_factor"], patience = config["patience"])
    # Training loop
    num_epochs = config["epochs"]
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_correct = 0
        total = 0
        correct_per_class = [0, 0, 0]  # For G1, G2, G3
        total_per_class = [0, 0, 0]  # For G1, G2, G3
        for batch in train_loader:
            frames = batch['frame'].float().to(device)
            labels = batch['label'].long().to(device)

            optimizer.zero_grad()
            outputs = model(frames)
            loss = criterion(outputs.logits, labels)
            _, predicted = torch.max(outputs.logits, 1)
            train_correct += (predicted == labels).sum().item()
            loss.backward()
            optimizer.step()
            total += labels.size(0)
            running_loss += loss.item()
            
            # Calculate accuracy per class
            for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                total_per_class[i] += (labels == i).sum().item()

        train_accuracy = 100 * train_correct / total
        class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")
        wandb.log({"Train Accuracy": train_accuracy, "Train loss": running_loss/len(train_loader), "G1_TrainAcc":class_accuracy[0], "G2_TrainAcc":class_accuracy[1], "G3_TrainAcc":class_accuracy[2]})

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        # Initialize counters for each class (G1, G2, G3)
        correct_per_class = [0, 0, 0]  # For G1, G2, G3
        total_per_class = [0, 0, 0]  # For G1, G2, G3

        with torch.no_grad():
            for batch in val_loader:
                frames = batch['frame'].float().to(device)
                labels = batch['label'].long().to(device)

                outputs = model(frames)
                loss = criterion(outputs.logits, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                # Calculate accuracy per class
                for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                    correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                    total_per_class[i] += (labels == i).sum().item()
        scheduler.step(val_loss)
        # Compute total accuracy and per-class accuracy
        total_accuracy = 100 * correct / total
        class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
        print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
        print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")
        # log metrics to wandb
        wandb.log({"Total Accuracy": total_accuracy, "Validation Loss": val_loss/len(val_loader), "G1_Acc":class_accuracy[0], "G2_Acc":class_accuracy[1], "G3_Acc":class_accuracy[2]})
    wandb.finish()  

In [8]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification

In [9]:
train_dataset = UnimodalCTDataset(split='train',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/" )
val_dataset = UnimodalCTDataset(split='val',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
print(f"Training set stats:{train_dataset.stats()}")
print(f"Validation set stats:{val_dataset.stats()}")

Training set stats:{'length': 2292, 'class_frequency': {'G1': 78, 'G2': 1543, 'G3': 671}}
Validation set stats:{'length': 335, 'class_frequency': {'G1': 37, 'G2': 166, 'G3': 132}}


In [11]:
labels = []
for sample in train_dataset:
    labels.append(sample["label"])
labels = np.array(labels)
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)

In [12]:
print(class_weights)

[9.79487179 0.49513934 1.13859911]


### Resnet-50

In [13]:
#model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
#model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [14]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [15]:
"""
config={
    "learning_rate": 1e-4,
    "architecture": "microsoft/resnet-50 new1",
    "epochs": 100,
    "weight_decay": 1e-4,
    "reduce_lr_factor": 0.2,
    "patience": 10,
    "class_weights": class_weights
    }
train(model, config, run_name = config["architecture"])
"""

'\nconfig={\n    "learning_rate": 1e-4,\n    "architecture": "microsoft/resnet-50 new1",\n    "epochs": 100,\n    "weight_decay": 1e-4,\n    "reduce_lr_factor": 0.2,\n    "patience": 10,\n    "class_weights": class_weights\n    }\ntrain(model, config, run_name = config["architecture"])\n'

### Resnet-18

In [16]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-18')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [18]:
config={
    "learning_rate": 5e-8,
    "architecture": "microsoft/resnet-18 not weighted",
    "epochs": 800,
    "weight_decay": 1e-6,
    "reduce_lr_factor": 0.25,
    "patience": 20,
    "class_weights": None
    }
train(model, config, run_name = config["architecture"])

wandb: Currently logged in as: pietro-caforio (pietro-caforio-politecnico-di-milano). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240919_161732-3ioql55n
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-18 not weighted
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/3ioql55n


Epoch 1, Loss: 1.0822495056523218
Validation Loss: 1.2490465884858912, Total Accuracy: 31.34%
Accuracy per class - G1: 0.00%, G2: 0.00%, G3: 79.55%
Epoch 2, Loss: 1.0650967823134527
Validation Loss: 1.220038278536363, Total Accuracy: 31.94%
Accuracy per class - G1: 0.00%, G2: 4.82%, G3: 75.00%
Epoch 3, Loss: 1.0544123558534517
Validation Loss: 1.2138974612409419, Total Accuracy: 31.94%
Accuracy per class - G1: 0.00%, G2: 4.82%, G3: 75.00%
Epoch 4, Loss: 1.0397988723384008
Validation Loss: 1.1985844427889043, Total Accuracy: 31.94%
Accuracy per class - G1: 0.00%, G2: 4.82%, G3: 75.00%
Epoch 5, Loss: 1.0268385377195146
Validation Loss: 1.1942293698137456, Total Accuracy: 31.94%
Accuracy per class - G1: 0.00%, G2: 4.82%, G3: 75.00%
Epoch 6, Loss: 1.017516246272458
Validation Loss: 1.1905517361380837, Total Accuracy: 31.64%
Accuracy per class - G1: 0.00%, G2: 4.22%, G3: 75.00%
Epoch 7, Loss: 1.0055798466006916
Validation Loss: 1.1850813681429082, Total Accuracy: 32.24%
Accuracy per class -

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:     G1_TrainAcc ▁▂▄▅▇▇██████████████████████████████████
wandb:          G2_Acc ▁▂▃▅▆▇▇█████████████████████████████████
wandb:     G2_TrainAcc ▁▄▆▆▇▇██████████████████████████████████
wandb:          G3_Acc ▆█████▇▇▅▇█▅▆▅▄▄▄▄▃▄▄▃▅▂▂▃▇▃▄▂▄▁▄▂▂▂▃▃▄▂
wandb:     G3_TrainAcc ▁▄▇▇████████████████████████████████████
wandb:  Total Accuracy ▁▂▄▅▆▇▇█▇██▇▇▇▇▇▇▇▇▇▇▇▇▆▆▇█▇▇▆▇▆▇▆▆▆▇▇▇▆
wandb:  Train Accuracy ▁▄▆▆▇▇██████████████████████████████████
wandb:      Train loss █▇▆▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: Validation Loss █▆▄▃▃▂▁▁▁▁▂▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▅▆▆
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0.0
wandb:     G1_TrainAcc 100.0
wandb:          G2_Acc 76.50602
wandb:     G2_TrainAcc 99.93519
wandb:          G3_Acc 62.12121
wandb:     G3_TrainAcc 100.0
wandb:  Total Accuracy 62.38806
wand

### Resnet-34

In [19]:
#model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
#model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [20]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [21]:
"""
config={
    "learning_rate": 1e-9,
    "architecture": "microsoft/resnet-34 weighted new",
    "epochs": 100,
    "weight_decay": 0.0,
    "reduce_lr_factor": 0.25,
    "patience": 20,
    "class_weights": class_weights
    }
train(model, config, run_name = config["architecture"])

"""


'\nconfig={\n    "learning_rate": 1e-9,\n    "architecture": "microsoft/resnet-34 weighted new",\n    "epochs": 100,\n    "weight_decay": 0.0,\n    "reduce_lr_factor": 0.25,\n    "patience": 20,\n    "class_weights": class_weights\n    }\ntrain(model, config, run_name = config["architecture"])\n\n'